In [1]:
# hide
import sys
if 'google.colab' in sys.modules: 
    !pip install -Uqq fastai transformers datasets
    !pip install -q git+git://github.com/walkwithfastai/walkwithfastai.github.io.git

In [2]:
#all_slow

# Text classification with transformers (intermediate)

> Fine-tuning pre-trained LM from HuggingFace model hub on GLUE benchmark

In [3]:
#hide
from wwf.utils import *

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from fastai.text.all import *

from datasets import load_dataset, concatenate_datasets
from inspect import signature
import gc

In [5]:
#hide_input
state_versions(['fastai', 'fastcore', 'transformers', 'datasets'])


---
This article is also a Jupyter Notebook available to be run from the top down. There
will be code snippets that you can then run in any environment.

Below are the versions of `fastai`, `fastcore`, `transformers`, and `datasets` currently running at the time of writing this:
* `fastai` : 2.3.1 
* `fastcore` : 1.3.19 
* `transformers` : 4.6.0 
* `datasets` : 1.6.1 
---

## Setup

In this notebook we will look at how to conbine the power of [HuggingFace](https://huggingface.co/) with great flexibility of [fastai](https://www.fast.ai/). For this purpose we will be finetuning `distilroberta-base` on The [General Language Understanding Evaluation(GLUE) benchmark](https://gluebenchmark.com/) tasks.

To give you a grasp on what are we dealing with, here is a brief summary of GLUE tasks:

In [6]:
#hide_input
abreviations=["cola","sst2","mrpc","stsb","qqp","mnli","qnli","rte","wnli"]
name = [
    "Corpus of Linguistic Acceptability",
    "Stanford Sentiment Treebank",
    "Microsoft Research Paraphrase Corpus",
    "Semantic Textual Similarity Benchmark",
    "Quora question pair",
    "Mulit-Genre Natural Language Inference",
    "Stanford Question Answering Dataset",
    "Recognize Textual Entailment",
    "Winograd Schema Challenge"
]
descriptions = [
    "Determine whether it is a grammatical sentence",
    "Predict the sentiment of a givensentence",
    "Determine whether the sentences in the pair are semantically equivalent",
    "Determine similarity score for 2 sentences",
    "Determine if 2 questions are the same (paraphrase)",
    "Predict whether the premise entails, contradicts or is neutral to the hypothesis",
    "Determine whether the context sentence containsthe answer to the question",
    "Determine whether one sentece entails another",
    "Predict if the sentence with the pronoun substituted is entailed by the original sentence"
]
df = pd.DataFrame({'Name':name,
                   'Task description':descriptions,
                   'Size':['8.5k','67k','3.7k','7k','364k','393k','105k','2.5k', '634'],
                   'Metrics':['matthews_corrcoef','accuracy','f1/accuracy','pearsonr/spearmanr',
                              'f1/accuracy','accuracy','accuracy','accuracy','accuracy']},
                   index=abreviations)
display_df(df)

,Name,Task description,Size,Metrics
cola,Corpus of Linguistic Acceptability,Determine whether it is a grammatical sentence,8.5k,matthews_corrcoef
sst2,Stanford Sentiment Treebank,Predict the sentiment of a givensentence,67k,accuracy
mrpc,Microsoft Research Paraphrase Corpus,Determine whether the sentences in the pair are semantically equivalent,3.7k,f1/accuracy
stsb,Semantic Textual Similarity Benchmark,Determine similarity score for 2 sentences,7k,pearsonr/spearmanr
qqp,Quora question pair,Determine if 2 questions are the same (paraphrase),364k,f1/accuracy
mnli,Mulit-Genre Natural Language Inference,"Predict whether the premise entails, contradicts or is neutral to the hypothesis",393k,accuracy
qnli,Stanford Question Answering Dataset,Determine whether the context sentence containsthe answer to the question,105k,accuracy
rte,Recognize Textual Entailment,Determine whether one sentece entails another,2.5k,accuracy
wnli,Winograd Schema Challenge,Predict if the sentence with the pronoun substituted is entailed by the original sentence,634,accuracy


Let's define main settings for the run in one place. You can choose any model from wide variety presented in HuggingFace model hub. Some might need special treatment to work but most models of appropriate class should be plug-and-play.

In [7]:
ds_name = 'glue'
model_name = "distilroberta-base"

max_len = 512
bs = 32
val_bs = bs*2

n_epoch = 4
lr = 2e-5
opt_func = Adam

To make switching between datasets smooth I'll define couple of dictionaries containing per-task information. We'll need metrics, text fields to retrieve data and number of outputs for the model.

In [8]:
GLUE_TASKS = ["cola", "mnli", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
def validate_task():
    assert task in GLUE_TASKS

In [9]:
glue_metrics = {
    'cola':[MatthewsCorrCoef()],
    'sst2':[accuracy],
    'mrpc':[F1Score(), accuracy],
    'stsb':[PearsonCorrCoef(), SpearmanCorrCoef()],
    'qqp' :[F1Score(), accuracy],
    'mnli':[accuracy],
    'qnli':[accuracy],
    'rte' :[accuracy],
    'wnli':[accuracy],
}

glue_textfields = {
    'cola':['sentence', None],
    'sst2':['sentence', None],
    'mrpc':['sentence1', 'sentence2'],
    'stsb':['sentence1', 'sentence2'],
    'qqp' :['question1', 'question2'],
    'mnli':['premise', 'hypothesis'],
    'qnli':['question', 'sentence'],
    'rte' :['sentence1', 'sentence2'],
    'wnli':['sentence1', 'sentence2'],
}

glue_num_labels = {'mnli':3, 'stsb':1}

## Data preprocessing

We'll be using `datasets` library for HuggingFace to get data:

In [10]:
task = 'mrpc'; validate_task()
ds = load_dataset(ds_name, task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


MNLI datasets contains 2 sets for validation: matched and missmatched. The mathced set is selected here for validation when fine-tuning on MNLI.

In [11]:
valid_ = 'validation-matched' if task=='mnli' else 'validation'
len(ds['train']), len(ds[valid_])

(3668, 408)

In [12]:
nt, nv = len(ds['train']), len(ds[valid_])
train_idx, valid_idx = L(range(nt)), L(range(nt, nt+nv))
train_ds = concatenate_datasets([ds['train'], ds[valid_]])

One can inspect single example for the given task:

In [13]:
train_ds[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

Here I use number of characters a proxy for length of tokenized text to speed up `dls` creation.

In [14]:
#hide_output
lens = train_ds.map(lambda s: {'len': sum([len(s[i]) for i in glue_textfields[task] if i])},
                    remove_columns=train_ds.column_names, num_proc=2, keep_in_memory=True)
train_lens = lens.select(train_idx)['len']
valid_lens = lens.select(valid_idx)['len']

## DataBlock and Transforms

`TextGetter` is analogous to `ItemGetter` but retrieves either one or two text fields from the source (e.g. "sentence1" and "sentence2").

In [15]:
class TextGetter(ItemTransform):
    def __init__(self, s1='text', s2=None):
        self.s1, self.s2 = s1, s2
    def encodes(self, sample):
        if self.s2 is None: return sample[self.s1]
        else: return sample[self.s1], sample[self.s2]

Transformers expect two parts of text to be concatenated with some `SEP` token inbetween. But when displaying the batch it's better to have those texts in separate columns for better readability. To make it work I define a version of `show_batch` to be dispatched on `TransTensorText` class. It will handle cases when there is single decoded text or a tuple of two texts.

In [16]:
class TransTensorText(TensorBase): pass

@typedispatch
def show_batch(x:TransTensorText, y, samples, ctxs=None, max_n=10, trunc_at=150, **kwargs):
    if ctxs is None: ctxs = get_empty_df(min(len(samples), max_n))
    if isinstance(samples[0][0], tuple):
        samples = L((*s[0], *s[1:]) for s in samples)
        if trunc_at is not None: samples = L((s[0].truncate(trunc_at), s[1].truncate(trunc_at), *s[2:]) for s in samples)
    if trunc_at is not None: samples = L((s[0].truncate(trunc_at),*s[1:]) for s in samples)
    ctxs = show_batch[object](x, y, samples, max_n=max_n, ctxs=ctxs, **kwargs)
    display_df(pd.DataFrame(ctxs))

In [17]:
#collapse
def find_first(t, e):
    for i, v in enumerate(t):
        if v == e: return i
        
def split_by_sep(t, sep_tok_id):
    idx = find_first(t, sep_tok_id)
    return t[:idx], t[idx:]

Tokenization of the inputs will be done by `TokBatchTransform` which wraps pre-trained HuggingFace tokenizer. The text processing is done in batches for speed-up. We want to awoid explicit python loops when possible.

In [18]:
class TokBatchTransform(Transform):
    def __init__(self, pretrained_model_name=None, tokenizer_cls=AutoTokenizer, 
                 config=None, tokenizer=None, with_labels=False,
                 padding=True, truncation=True, max_length=None, **kwargs):
        if tokenizer is None:
            tokenizer = tokenizer_cls.from_pretrained(pretrained_model_name, config=config)
        self.tokenizer = tokenizer
        self.kwargs = kwargs
        self._two_texts = False
        store_attr()
    
    def encodes(self, batch):
        # batch is a list of tuples of ({text or (text1, text2)}, {targets...})
        if is_listy(batch[0][0]): # 1st element is tuple
            self._two_texts = True
            texts = ([s[0][0] for s in batch], [s[0][1] for s in batch])
        elif is_listy(batch[0]): 
            texts = ([s[0] for s in batch],)

        inps = self.tokenizer(*texts,
                              add_special_tokens=True,
                              padding=self.padding,
                              truncation=self.truncation,
                              max_length=self.max_length,
                              return_tensors='pt',
                              **self.kwargs)
        # inps are batched, collate targets into batches too
        labels = default_collate([s[1:] for s in batch])
        if self.with_labels:
            inps['labels'] = labels[0]
            res = (inps, )
        else:
            res = (inps, ) + tuple(labels)
        return res
    
    def decodes(self, x:TransTensorText):
        if self._two_texts:
            x1, x2 = split_by_sep(x, self.tokenizer.sep_token_id)
            return (TitledStr(self.tokenizer.decode(x1.cpu(), skip_special_tokens=True)),
                    TitledStr(self.tokenizer.decode(x2.cpu(), skip_special_tokens=True)))
        return TitledStr(self.tokenizer.decode(x.cpu(), skip_special_tokens=True))

The batches processed by `TokBatchTransform` contain a dictionary as the first element. For decoding it's handy to have a tensor instead. The `Undict` transform fethces `input_ids` from the batch and creates `TransTensorText` which should work with typedispatch.

In [19]:
class Undict(Transform):
    def decodes(self, x:dict):
        if 'input_ids' in x: res = TransTensorText(x['input_ids'])
        return res

Now the transforms are to be combined inside a data block to be used for `dls` creation. The inputs are prebatched by `TokBatchTranform` so we don't need to use `fa_collate` for batching, so `fa_convert` is passed in as for "create_batch".

In [20]:
dls_kwargs = {
    'before_batch': TokBatchTransform(pretrained_model_name=model_name, max_length=max_len),
    'create_batch': fa_convert
}
text_block = TransformBlock(dl_type=SortedDL, dls_kwargs=dls_kwargs, batch_tfms=Undict(), )

dblock = DataBlock(blocks = [text_block, CategoryBlock()],
                   get_x=TextGetter(*glue_textfields[task]),
                   get_y=ItemGetter('label'),
                   splitter=IndexSplitter(valid_idx))

In [21]:
%%time
dl_kwargs=[{'res':train_lens}, {'val_res':valid_lens}]
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=val_bs, dl_kwargs=dl_kwargs)

CPU times: user 2.74 s, sys: 892 ms, total: 3.63 s
Wall time: 3.66 s


In [22]:
dls.show_batch(max_n=4)

,text,text_,category
0,The Securities and Exchange Commission yesterday said companies trading on the biggest U.S. markets must win shareholder approval before granting stock options and other stock-based compensation plans to corporate executives.,Companies trading on the biggest stock markets must get shareholder approval before granting stock options and other equity compensation under rules cleared yesterday by the Securities and Exchange Commission.,1
1,"The report by the Justice Department's inspector general found "" significant problems "" in the government's handling of foreigners who were jailed under blanket edicts adopted by the department after the attacks.","The Justice Department's Office of the Inspector General described "" significant problems "" in the Bush administration's actions toward the 762 foreigners held on immigration violations after the attacks.",1
2,"The government said FirstEnergy Nuclear determined that a contractor had established an unprotected high-speed computer connection to its corporate network that allowed the "" Slammer "" infection to spread internally.",It said FirstEnergy determined that a contractor had established an unprotected computer connection to its corporate network that allowed the so-called ``Slammer'' worm to spread internally.,1
3,"Microsoft favors setting up "" independent e-mail trust authorities to establish and maintain commercial email guidelines, certify senders who follow the guidelines, and resolve customer disputes. ""","Gates says he wants to see "" independent e-mail trust authorities "" who "" establish and maintain commercial email guidelines, certify senders who follow the guidelines, and resolve customer disputes. """,1


## Customised Learner

Now the `xb` we get from dataloader contains a dictionary and HuggingFace transformers accept keyword argument as input. To make this work smoothly I choose to create `Learner` subclass to handle correct unrolling of the input dict.

But first we need to define some utility functions. `default_splitter` is used to devide model parameters into groups:

In [23]:
def default_splitter(model):
    groups = L(model.base_model.children()) + L(m for m in list(model.children())[1:] if params(m))
    return groups.map(params)

Similar to `show_batch` one have to customize `show_results`:

In [24]:
@typedispatch
def show_results(x: TransTensorText, y, samples, outs, ctxs=None, max_n=10, trunc_at=150, **kwargs):
    if ctxs is None: ctxs = get_empty_df(min(len(samples), max_n))
    if isinstance(samples[0][0], tuple):
        samples = L((*s[0], *s[1:]) for s in samples)
        if trunc_at is not None: samples = L((s[0].truncate(trunc_at), s[1].truncate(trunc_at), *s[2:]) for s in samples)
    elif trunc_at is not None: samples = L((s[0].truncate(trunc_at),*s[1:]) for s in samples)
    ctxs = show_results[object](x, y, samples, outs, ctxs=ctxs, max_n=max_n, **kwargs)
    display_df(pd.DataFrame(ctxs))
    return ctxs

`TransLearner` itself doesn't do much. `_do_one_batch` method is reworked to feed the unrolled input dict to the model after filtering unexpected keys. It also takes care of some small things like adding `TransCallback` and setting `splitter` to be `default_splitter` if none is provided.

In [25]:
@delegates(Learner.__init__)
class TransLearner(Learner):
    "Learner for training transformers from HuggingFace"
    def __init__(self, dls, model, **kwargs):
        splitter = kwargs.get('splitter', None)
        if splitter is None: kwargs['splitter'] = default_splitter
        super().__init__(dls, model, **kwargs)
        self.model_args = set(signature(model.forward).parameters.keys())
        self.add_cb(TransCallback())
        self.compute_loss = True

    def one_batch(self, i, b):
        self.iter = i
        b_on_device = tuple(to_device(e) for e in b) if self.dls.device is not None else b
        self._split(b_on_device)
        self._with_events(self._do_one_batch, 'batch', CancelBatchException)
    
    def _do_one_batch(self):
        x = self.xb[0]
        for k in x.keys():
            if k not in self.model_args: del x[k]
        self.pred = self.model(**self.x)
        self('after_pred')
        if len(self.yb) and self.compute_loss:
            self.loss_grad = self.loss_func(self.pred, *self.yb)
            self.loss = self.loss_grad.clone()
        self('after_loss')
        if not self.training or not len(self.yb): return
        self('before_backward')
        self.loss_grad.backward()
        self._with_events(self.opt.step, 'step', CancelStepException)
        self.opt.zero_grad()

By default the model returns dictionary-like object containing `logits` and possibly other outputs as defined by model config (e.g. intermediate hidden representations). In fastai training loop we usually expect `preds` to be a tensor containing model predictions (logits). To format the preds properly we can use a callback.

Notice that if `labels` key is found in the input, transformer models compute the `loss` and return it togather with output `logits`. The callback below is designed to utilise the loss returned by model instead of recomputing it using `learn.loss_func`. This is not actually used in this example but might be handy in some usecases.

In [26]:
class TransCallback(Callback):
    "Handles HuggingFace model outputs"
    def after_pred(self):
        if 'loss' in self.pred:
            self.learn.loss_grad = self.pred.loss
            self.learn.loss = self.pred.loss.clone()
            if 'labels' in self.xb[0].keys():
                self.learn.yb = (self.xb[0]['labels'], )
            self.learn.compute_loss = False
        self.learn.pred = self.pred.logits

## Training

After all the preparations the training is streightforward. Setting `num_labels` for the model and choosing apropriate metrics is automated.

In [27]:
#hide_output
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=glue_num_labels.get(task, 2))
metrics = glue_metrics[task]
learn = TransLearner(dls, model, metrics=metrics, opt_func=opt_func)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [28]:
#collapse_output
learn.summary()

RobertaForSequenceClassification (Input shape: 32)
Layer (type)         Output Shape         Param #    Trainable 
                     32 x 103 x 768      
Embedding                                 38603520   True      
Embedding                                 394752     True      
Embedding                                 768        True      
LayerNorm                                 1536       True      
Dropout                                                        
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
Dropout                                                        
Linear                                    590592     True      
LayerNorm                                 1536       True      
Dropout                                                        
___________________________________________________________________________

In [29]:
metric_to_monitor = metrics[0].name if isinstance(metrics[0], Metric) else metrics[0].__name__
cbs = [SaveModelCallback(monitor=metric_to_monitor)]
learn.fit_one_cycle(4, lr, cbs=cbs)

epoch,train_loss,valid_loss,f1_score,accuracy,time
0,0.610563,0.538350,0.819970,0.708333,00:20
1,0.455006,0.353764,0.889279,0.845588,00:21
2,0.320432,0.384724,0.888519,0.835784,00:20
3,0.238442,0.350724,0.897747,0.855392,00:20


Better model found at epoch 0 with f1_score value: 0.8199697428139183.
Better model found at epoch 1 with f1_score value: 0.8892794376098417.
Better model found at epoch 3 with f1_score value: 0.8977469670710572.


After training the model it's useful to verify  that results make sense:

In [30]:
learn.show_results()

,text,text_,category,category_
0,"The delegates said raising and distributing funds has been complicated by the U.S. crackdown on jihadi charitable foundations, bank accounts of terror-related organizations and money transfers.","Bin Laden ’ s men pointed out that raising and distributing funds has been complicated by the U.S. crackdown on jihadi charitable foundations, bank accounts of terror-related organizations and money transfers.",1,1
1,The attack followed several days of disturbances in the city where American soldiers exchanged fire with an unknown number of attackers as civilians carried out demonstrations against the American presence.,The attack came after several days of disturbance in the city in which U.S. soldiers exchanged fire with an unknown number of attackers as civilians protested the American presence.,1,1
2,Massachusetts regulators and the Securities and Exchange Commission on Tuesday pressed securities fraud charges against Putnam Investments and two of its former portfolio managers for alleged improper mutual fund trading.,State and federal securities regulators filed civil charges against Putnam Investments and two portfolio managers in the ever-expanding mutual fund trading scandal.,1,1
3,Justice Minister Martin Cauchon and Prime Minister Jean Chrétien have both said the Liberal government will introduce legislation soon to decriminalize possession of small amounts of pot for personal use.,Justice Minister Martin Cauchon and Prime Minister Jean Chretien both have said the government will introduce legislation to decriminalize possession of small amounts of pot.,1,1
4,"Myanmar's pro-democracy leader Aung San Suu Kyi will return home late Friday but will remain in detention after recovering from surgery at a Yangon hospital, her personal physician said.","Myanmar's pro-democracy leader Aung San Suu Kyi will be kept under house arrest following her release from a hospital where she underwent surgery, her personal physician said Friday.",1,1
5,"President Bush raised a record-breaking $ 49.5 million for his re-election campaign over the last three months, with contributions from 262,000 Americans, the president's campaign chairman said Tuesday.","President Bush has raised $ 83.9 million since beginning his re-election campaign in May, and has $ 70 million of that left to spend, his campaign said Tuesday.",0,0
6,"Barry Callebaut will be able to use Brach's retail network to sell products made from its German subsidiary Stollwerck, which makes chocolate products not sold in the United States.","Barry Callebaut will be able to use Brach's retail network to sell products made from its German subsidiary Stollwerck, which makes chocolate products unknown to the American market.",1,1
7,"The Guru microcontroller serves four functions : hardware monitoring, overclocking management, BIOS ( Basic Input Output System ) update and a troubleshooting-assistance feature called Black Box.","The µGuru microcontroller serves four functions : hardware monitoring, overclocking management, BIOS update and a troubleshooting-assistance feature called Black Box.",1,1
8,"Donations stemming from the Sept. 11 attacks helped push up contributions to human service organizations and large branches of the United Way by 15 percent and 28.6 percent, respectively.",Donations stemming from the Sept. 11 attacks helped push up contributions to human service organizations by 15 percent and to large branches of the United Way by 28.6 percent.,1,1


Finally we can run our model on test set to get the predictoins.

In [31]:
test_dl = dls.test_dl(ds['test'])
preds = learn.get_preds(dl=test_dl)
preds[0]

tensor([[0.0094, 0.9906],
        [0.0475, 0.9525],
        [0.0031, 0.9969],
        ...,
        [0.0195, 0.9805],
        [0.0025, 0.9975],
        [0.0151, 0.9849]])

## Final remarks

Generalised versions of "wrapper" code used in this notebook can be found in [fasthugs](https://github.com/aikindergarten/fasthugs) library. Also you can check out some extra info on fine-tuning models on GLUE tasks in [this blogpost](https://arampacha.github.io/thoughtsamples/fastai/huggingface/transformers/2021/05/07/glue-benchmark.html).